https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/01-basics/pytorch_basics/main.py

# Basic pytorch

In [3]:
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms


# ================================================================== #
#                         Table of Contents                          #
# ================================================================== #

# 1. Basic autograd example 1               (Line 25 to 39)
# 2. Basic autograd example 2               (Line 46 to 83)
# 3. Loading data from numpy                (Line 90 to 97)
# 4. Input pipline                          (Line 104 to 129)
# 5. Input pipline for custom dataset       (Line 136 to 156)
# 6. Pretrained model                       (Line 163 to 176)
# 7. Save and load model                    (Line 183 to 189) 


In [4]:
# ================================================================== #
#                     1. Basic autograd example 1                    #
# ================================================================== #

# Create tensors.
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

# Build a computational graph.
y = w * x + b    # y = 2 * x + 3

# Compute gradients.
y.backward()

# Print out the gradients.
print(x.grad)    # x.grad = 2 
print(w.grad)    # w.grad = 1 
print(b.grad)    # b.grad = 1 


tensor(2.)
tensor(1.)
tensor(1.)


In [5]:
# ================================================================== #
#                    2. Basic autograd example 2                     #
# ================================================================== #

# Create tensors of shape (10, 3) and (10, 2).
x = torch.randn(10, 3)
y = torch.randn(10, 2)

# Build a fully connected layer.
linear = nn.Linear(3, 2)
print ('w: ', linear.weight)
print ('b: ', linear.bias)

# Build loss function and optimizer.
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

for i in range(100):
    # Forward pass.
    pred = linear(x)

    # Compute loss.
    loss = criterion(pred, y)
    print(i, 'loss: ', loss.item())

    # Backward pass.
    loss.backward()

    # Print out the gradients.
    #print ('dL/dw: ', linear.weight.grad) 
    #print ('dL/db: ', linear.bias.grad)

    # 1-step gradient descent.
    optimizer.step()
    optimizer.zero_grad()

# You can also perform gradient descent at the low level.
# linear.weight.data.sub_(0.01 * linear.weight.grad.data)
# linear.bias.data.sub_(0.01 * linear.bias.grad.data)

# Print out the loss after 1-step gradient descent.
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.item())

w:  Parameter containing:
tensor([[-0.1342, -0.1649,  0.3521],
        [ 0.3312,  0.2525,  0.0676]], requires_grad=True)
b:  Parameter containing:
tensor([ 0.0650, -0.3146], requires_grad=True)
0 loss:  1.9331130981445312
1 loss:  1.87247633934021
2 loss:  1.8150007724761963
3 loss:  1.7605139017105103
4 loss:  1.7088534832000732
5 loss:  1.6598657369613647
6 loss:  1.6134058237075806
7 loss:  1.5693362951278687
8 loss:  1.527528166770935
9 loss:  1.4878584146499634
10 loss:  1.450211763381958
11 loss:  1.41447913646698
12 loss:  1.3805569410324097
13 loss:  1.3483476638793945
14 loss:  1.3177595138549805
15 loss:  1.288704752922058
16 loss:  1.2611016035079956
17 loss:  1.2348718643188477
18 loss:  1.20994234085083
19 loss:  1.1862432956695557
20 loss:  1.1637089252471924
21 loss:  1.142277717590332
22 loss:  1.1218903064727783
23 loss:  1.1024916172027588
24 loss:  1.0840290784835815
25 loss:  1.066453456878662
26 loss:  1.0497174263000488
27 loss:  1.0337769985198975
28 loss:  1.018

In [6]:
# ================================================================== #
#                     3. Loading data from numpy                     #
# ================================================================== #

# Create a numpy array.
x = np.array([[1, 2], [3, 4]])

# Convert the numpy array to a torch tensor.
y = torch.from_numpy(x)
#y = torch.tensor(x)

# Convert the torch tensor to a numpy array.
z = y.numpy()

In [7]:
torch.cuda.is_available()

False

In [8]:
x = np.random.randn(100000)
y = np.random.randn(100000)
%timeit x.dot(y)

48 µs ± 16.8 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [9]:
x = torch.tensor(np.random.randn(100000)).to('cpu')
y = torch.tensor(np.random.randn(100000)).to('cpu')
%timeit x.dot(y)

39.7 µs ± 2.64 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [10]:
x = torch.tensor(np.random.randn(100000)).to('cuda')
y = torch.tensor(np.random.randn(100000)).cuda()
%timeit x.dot(y)

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
# ================================================================== #
#                         4. Input pipeline                           #
# ================================================================== #

# Download and construct CIFAR-10 dataset.
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform=transforms.ToTensor(),
                                             download=True)

# Fetch one data pair (read data from disk).
image, label = train_dataset[0]
print (image.size())
print (label)

# Data loader (this provides queues and threads in a very simple way).
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64, 
                                           shuffle=True)

# When iteration starts, queue and thread start to load data from files.
data_iter = iter(train_loader)

# Mini-batch images and labels.
images, labels = data_iter.next()

# Actual usage of the data loader is as below.
for images, labels in train_loader:
    # Training code should be written here.
    pass


In [ ]:
x = torch.randn(100, 5)
y = torch.randn(100, 1)
dataset = torch.utils.data.TensorDataset(x, y)
loader = torch.utils.data.DataLoader(dataset=dataset,
                                           batch_size=10, 
                                           shuffle=False)

In [ ]:
for x_batch, y_batch in loader:
    print(x_batch)
    print(y_batch)
    break

In [ ]:
dataset[0], x[0], y[0]

In [ ]:
# ================================================================== #
#                5. Input pipeline for custom dataset                 #
# ================================================================== #
import os
# You should build your custom dataset as below.
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, folder_name):
        # TODO
        # 1. Initialize file paths or a list of file names. 
        self.folder_name = folder_name
        self.files = os.listdir(self.folder_name)
        
    def __getitem__(self, index):
        # TODO
        # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
        # 2. Preprocess the data (e.g. torchvision.Transform).
        # 3. Return a data pair (e.g. image and label).
        return self.files[index]
    
    def __len__(self):
        # You should change 0 to the total size of your dataset.
        return len(self.files)

# You can then use the prebuilt data loader. 
custom_dataset = CustomDataset(folder_name='.')
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                           batch_size=4, 
                                           shuffle=True)


In [ ]:
custom_dataset[1]

In [ ]:
for names in train_loader:
    print(names)
    break

In [ ]:
a = [1,2,3]
len(a), a.__len__()

In [ ]:
dir(a)
a[1], a.__getitem__(1)

In [ ]:
# ================================================================== #
#                        6. Pretrained model                         #
# ================================================================== #

# Download and load the pretrained ResNet-18.
resnet = torchvision.models.resnet18(pretrained=True)

# If you want to finetune only the top layer of the model, set as below.
for param in resnet.parameters():
    param.requires_grad = False

# Replace the top layer for finetuning.
resnet.fc = nn.Linear(resnet.fc.in_features, 100)  # 100 is an example.

# Forward pass.
images = torch.randn(64, 3, 224, 224)
outputs = resnet(images)
print (outputs.size())     # (64, 100)

In [ ]:
# ================================================================== #
#                      7. Save and load the model                    #
# ================================================================== #

# Save and load the entire model.
torch.save(resnet, 'model.ckpt')
model = torch.load('model.ckpt')

# Save and load only the model parameters (recommended).
torch.save(resnet.state_dict(), 'params.ckpt')
resnet.load_state_dict(torch.load('params.ckpt'))